In [ ]:
!pip install nltk==3.4.5
import nltk.lm
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re
import pickle

     |████████████████████████████████| 1.5MB 4.4MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=e950e4785a33b42223231d9c747dab0bcba63292fe46ffb93322dceb295faccc
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
topic = "SPORTS"
with open('./drive/MyDrive/4rd1/DataMining/IRTM_Final/{}_Raw.pkl'.format(topic), "rb") as fp:
  train = pickle.load(fp)

In [ ]:
all_text = ""
N = len(train)
for i in range(N):
  all_text += train[i]['content']

In [ ]:
(train[0]['content'])

'When it comes to New York Jets players kneeling during the national anthem, team chairman Christopher Johnson is a standup guy.Johnson said the team wouldn’t fine any player for protesting, despite a new NFL owner-approved measure that enables teams to do so, according to reports.Wednesday’s rule change says\xa0players must stand for the anthem or stay off the field. If any player kneels, his team would face a league fine. Teams can discipline individual players.“As I have in the past, I will support our players wherever we land as a team,” Johnson said in a statement, according to the New York Post. “Our focus is not on imposing any Club rules, fines, or restrictions. Instead we will continue to work closely with our players to constructively advance social justice issues that are important to us. I remain extremely proud of how we demonstrated unity last season as well as our players’ commitment to strengthening our communities.”Johnson elaborated on his pledge not to fine players\x

In [ ]:
data_text = all_text
def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)

In [ ]:
import markovify
print(train[0]['content'])

When it comes to New York Jets players kneeling during the national anthem, team chairman Christopher Johnson is a standup guy.Johnson said the team wouldn’t fine any player for protesting, despite a new NFL owner-approved measure that enables teams to do so, according to reports.Wednesday’s rule change says players must stand for the anthem or stay off the field. If any player kneels, his team would face a league fine. Teams can discipline individual players.“As I have in the past, I will support our players wherever we land as a team,” Johnson said in a statement, according to the New York Post. “Our focus is not on imposing any Club rules, fines, or restrictions. Instead we will continue to work closely with our players to constructively advance social justice issues that are important to us. I remain extremely proud of how we demonstrated unity last season as well as our players’ commitment to strengthening our communities.”Johnson elaborated on his pledge not to fine players in an

In [ ]:
text_model = markovify.Text(train[0]['content'], state_size=2)
print(train[0].keys())

dict_keys(['category', 'headline', 'authors', 'link', 'short_description', 'date', 'content'])


In [ ]:
print(train[0]['headline'])
for i in range(5):
  print(text_model.make_sentence())

Jets Chairman Christopher Johnson Won't Fine Players For Anthem Protests
“You have to stand proudly for the national anthem or stay off the field.
Teams can discipline individual players.“As I have in the past, I will support our players are on the front lines.”Johnson’s stance could get interesting in the team’s hierarchy.
Teams can discipline individual players.“As I have in the past, I will support our players are on the front lines.”Johnson’s stance could get interesting in the team’s hierarchy.
Teams can discipline individual players.“As I have in the past, I will support our players are on the front lines.”Johnson’s stance could get interesting in the team’s hierarchy.
“Our focus is not on imposing any club-specific rules,” he said.


In [ ]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)

Total Sequences: 9120058


In [ ]:
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [ ]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (8208052, 30) Val shape: (912006, 30)


In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=10, batch_size = 3000,verbose=1, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 50)            1350      
_________________________________________________________________
gru (GRU)                    (None, 150)               90900     
_________________________________________________________________
dense (Dense)                (None, 27)                4077      
Total params: 96,327
Trainable params: 96,327
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
2500/2737 [==========================>...] - ETA: 8:06 - loss: 1.8324 - acc: 0.4485

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

In [ ]:
t = generate_seq(model, mapping, 15, "", 100)
print(t)

In [ ]:
!git clone https://github.com/huggingface/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 59307, done.
remote: Total 59307 (delta 0), reused 0 (delta 0), pack-reused 59307
Receiving objects: 100% (59307/59307), 44.34 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (41758/41758), done.


In [ ]:
!python pytorch-transformers/examples/run_generation.py --model_type=gpt2 --length=100 --model_name_or_path=gpt2

python3: can't open file 'pytorch-transformers/examples/run_generation.py': [Errno 2] No such file or directory


In [ ]:
!git clone https://github.com/nlpyang/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 6469, done.
remote: Total 6469 (delta 0), reused 0 (delta 0), pack-reused 6469
Receiving objects: 100% (6469/6469), 3.60 MiB | 12.34 MiB/s, done.
Resolving deltas: 100% (4641/4641), done.


In [ ]:
# 預測出一整個文章
!python pytorch-transformers/examples/run_generation.py --model_type=gpt2 --length=60 --model_name_or_path=gpt2

01/17/2021 03:14:35 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json not found in cache or force_download set to True, downloading to /tmp/tmpt9z2e0c9
100% 1042301/1042301 [00:00<00:00, 12378324.84B/s]
01/17/2021 03:14:35 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpt9z2e0c9 to cache at /root/.cache/torch/pytorch_transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
01/17/2021 03:14:35 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
01/17/2021 03:14:35 - INFO - pytorch_transformers.file_utils -   removing temp file /tmp/tmpt9z2e0c9
01/17/2021 03:14:35 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.hu

In [ ]:
!pip install pytorch_transformers

     |████████████████████████████████| 184kB 4.3MB/s 
     |████████████████████████████████| 890kB 7.4MB/s 
     |████████████████████████████████| 133kB 12.8MB/s 
     |████████████████████████████████| 1.2MB 13.3MB/s 
     |████████████████████████████████| 7.2MB 20.5MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0fd8b159756dc54e165d742d2c9ff4c8390d417e5660815ab1857853187a1ba8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.56 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!git clone https://github.com/nlpyang/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 6469, done.
remote: Total 6469 (delta 0), reused 0 (delta 0), pack-reused 6469
Receiving objects: 100% (6469/6469), 3.60 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (4641/4641), done.


In [ ]:
# 預測下一個字

import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# 加载预训练模型tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 对文本输入进行编码
text = "What is the fastest car in the"
text = "I want to fuck"
indexed_tokens = tokenizer.encode(text)

# 在PyTorch张量中转换indexed_tokens
tokens_tensor = torch.tensor([indexed_tokens])

# 加载预训练模型 (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

#将模型设置为evaluation模式，关闭DropOut模块
model.eval()

# 如果你有GPU，把所有东西都放在cuda上
# tokens_tensor = tokens_tensor.to('cuda')
# model.to('cuda')

# 预测所有的tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# 得到预测的单词
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

# 打印预测单词
print(predicted_text)

 I want to fuck you
